### json file 관련 함수 

- save_json : 데이터를 json으로 저장
- load_json : json 파일 불러오기
- 기본 경로 현재 폴더인 'data'로 지정 해둠.

In [8]:
import os
import json

def save_json(data, filename):
    with open(filename, 'w', encoding='utf-8') as f:
        json.dump(data, f, indent=4, ensure_ascii=False)

def load_json(filename):
    if not os.path.exists(filename):
        return None
    with open(filename, 'r', encoding='utf-8') as f:
        return json.load(f)

### 마켓컬리 식품 항목 상품 크롤링

- 큰 카테고리 : "대분류", 그 다음 카테고리 : "소분류" 라고 정의

In [ ]:
import time
import logging
import json
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import NoSuchElementException
from webdriver_manager.chrome import ChromeDriverManager

# 로깅 설정
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s [%(levelname)s] : %(message)s',
    handlers=[logging.StreamHandler()]
)

# 대분류 카테고리 ID 리스트
CATEGORY_IDS = ['907', '908', '909', '910', '911', '912', '913', '914', '383', '249', '915', '018', '032', '722', '251']

# 크롤링 함수
def crawl_products_kurly():
    options = webdriver.ChromeOptions()
    options.add_argument('--headless') # gui 미사용 
    options.add_argument('--start-maximized') # 전체화면
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

    all_products = []

    try:
        for category_id in CATEGORY_IDS:
            main_base_url = f'https://www.kurly.com/categories/{category_id}'
            driver.get(main_base_url)
            time.sleep(2)

            try:
                main_category = driver.find_element(By.XPATH, '//*[@id="container"]/h3').text.strip()
                logging.info(f'대분류({category_id}) - {main_category}')

                sub_category_count = len(driver.find_elements(By.CSS_SELECTOR, '#container > ul > li'))
                for i in range(2, sub_category_count + 1):
                    try:
                        sub_xpath = f'//*[@id="container"]/ul/li[{i}]/a'
                        sub_element = driver.find_element(By.XPATH, sub_xpath)
                        sub_category = sub_element.text.strip()
                        logging.info(f'소분류: {sub_category}')

                        sub_element.click()
                        time.sleep(2)

                        sub_base_url = driver.current_url.split('&page=')[0]
                        page = 1

                        while True:
                            page_url = f"{sub_base_url}&page={page}"
                            driver.get(page_url)
                            time.sleep(2)

                            if "상품이 없습니다" in driver.page_source:
                                logging.info("더 이상 상품 없음.")
                                break

                            logging.info(f'{page}페이지 수집 중...')

                            if sub_category.startswith("더퍼플"):
                                product_xpath_base = '//*[@id="container"]/div[3]/div[2]/div[2]/a'
                            else:
                                product_xpath_base = '//*[@id="container"]/div[2]/div[2]/div[2]/a'

                            product_elements = driver.find_elements(By.XPATH, product_xpath_base)

                            for idx in range(len(product_elements)):
                                try:
                                    product_elements = driver.find_elements(By.XPATH, product_xpath_base)
                                    product = product_elements[idx]
                                    driver.execute_script("arguments[0].scrollIntoView();", product)
                                    product.click()
                                    time.sleep(2)

                                    product_url = driver.current_url
                                    product_name = driver.find_element(
                                        By.XPATH,
                                        '//*[@id="product-atf"]/section/div[1]/div[1]/div[2]/h1'
                                    ).text.strip()

                                    # 기존에 있는지 확인(상품 URL 기준)
                                    existing = next((item for item in all_products if item['상품 url'] == product_url), None)

                                    if existing:
                                        if sub_category not in existing['소분류']:
                                            existing['소분류'].append(sub_category)
                                    else:
                                        all_products.append({
                                            '상품명': product_name,
                                            '상품 url': product_url,
                                            '대분류': main_category,
                                            '소분류': [sub_category]
                                        })

                                    driver.back()
                                    time.sleep(2)

                                except Exception as e:
                                    logging.warning(f'상품 오류: {e}')
                                    driver.get(page_url)
                                    time.sleep(1)

                            page += 1

                        # 소분류 하나 끝날 때 저장
                        save_json(all_products, "products_kurly.json")
                        logging.info(f'현재까지 저장된 상품 수: {len(all_products)}')

                        driver.get(main_base_url)
                        time.sleep(2)

                    except Exception as e:
                        logging.error(f'소분류 {i} 오류: {e}')
                        driver.get(main_base_url)
                        time.sleep(2)

            except Exception as e:
                logging.error(f'대분류 {category_id} 오류: {e}')

    finally:
        driver.quit()

    return all_products

# 실행
if __name__ == "__main__":
    result = crawl_products_kurly()
    save_json(result, "products_kurly.json")
    logging.info(f'전체 크롤링 완료 - 총 {len(result)}개 상품 저장됨')


2025-06-12 10:32:14,028 [INFO] : ====== WebDriver manager ======
2025-06-12 10:32:15,082 [INFO] : Get LATEST chromedriver version for google-chrome
2025-06-12 10:32:15,122 [INFO] : Get LATEST chromedriver version for google-chrome
2025-06-12 10:32:15,162 [INFO] : Driver [C:\Users\User\.wdm\drivers\chromedriver\win64\137.0.7151.70\chromedriver.exe] found in cache


KeyboardInterrupt: 